In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML,clear_output
import joblib

In [2]:
def readData_basicExtract():
    purchasedf = pd.read_csv('../input/buyinghabitsofdigikalausers/purchase_history.csv')
    purchasedf["created_at"] = pd.to_datetime(purchasedf["created_at"])
    
    sortedPurchasedf = purchasedf.sort_values(by=['user_id','product_id','created_at'])
    del purchasedf
    print('purchase shape dataframe shape: ',sortedPurchasedf.shape)
    display(HTML(sortedPurchasedf.head().to_html()))
    
    ALLdata = sortedPurchasedf.copy()
    ALLdata['diffDay'] = ALLdata.groupby(['user_id','product_id'],as_index=False)['created_at'].transform(lambda x : x-x.shift(1))
    ALLdata['diffDay'] = ALLdata['diffDay'].dt.days
    firstRecord = pd.Timestamp('2019-01-01')
    ALLdata['GoneDays'] = (ALLdata['created_at'] - firstRecord).dt.days
    ALLdata.reset_index(inplace=True)
    print('purchase df with diffDay shape: ',ALLdata.shape)
    display(HTML(ALLdata.tail(20).to_html()))
    print('-----------------------------------------------------------------')
    target = ALLdata.reset_index().groupby(['user_id','product_id'],as_index=False).agg(created_at=('created_at','max'),
                                                                     diffDay=('diffDay',lambda x:x.iloc[-1]),
                                                                     GoneDays=('GoneDays',lambda x:x.iloc[-1]),
                                                                      index = ('index',lambda x:x.iloc[-1]) )
    print('target(last purchase datetime) shape: ',target.shape)
    display(HTML(target.tail(5).to_html()))
    trainData = ALLdata[~ALLdata['index'].isin(target['index'])]
    print('trainData shape: ',trainData.shape)
    display(HTML(trainData.tail(20).to_html()))
    
    return ALLdata,trainData,target

ALLdata,trainData,target = readData_basicExtract()

purchase shape dataframe shape:  (2174447, 3)


,user_id,product_id,created_at
68489,577243,724112,2019-03-13
1304270,577243,724112,2019-03-25
418727,577243,724112,2019-04-22
1859117,577243,724112,2019-06-16
988009,577243,724112,2019-06-29


purchase df with diffDay shape:  (2174447, 6)


,index,user_id,product_id,created_at,diffDay,GoneDays
2174427,349309,45443053,1660612,2019-12-07,1.0,340
2174428,637531,45443053,1660612,2019-12-24,17.0,357
2174429,919997,45443053,1660612,2019-12-24,0.0,357
2174430,1214103,45443053,1660612,2019-12-24,0.0,357
2174431,1352989,45443053,1660612,2019-12-24,0.0,357
2174432,580497,45471482,1945563,2019-02-17,NaN,47
2174433,483565,45471482,1945563,2019-02-27,10.0,57
2174434,1309264,45471482,1945563,2019-03-17,18.0,75
2174435,964042,45471482,1945563,2019-08-10,146.0,221
2174436,320355,45471482,3309954,2019-10-31,NaN,303


-----------------------------------------------------------------
target(last purchase datetime) shape:  (318050, 6)


,user_id,product_id,created_at,diffDay,GoneDays,index
318045,45431507,4375282,2019-08-27,47.0,238,806653
318046,45443053,1660612,2019-12-24,0.0,357,1352989
318047,45471482,1945563,2019-08-10,146.0,221,964042
318048,45471482,3309954,2019-11-15,4.0,318,1150243
318049,45471482,4563945,2019-11-15,4.0,318,759240


trainData shape:  (1856397, 6)


,index,user_id,product_id,created_at,diffDay,GoneDays
2174423,1435673,45443053,1660612,2019-11-28,1.0,331
2174424,395274,45443053,1660612,2019-11-30,2.0,333
2174425,2103062,45443053,1660612,2019-12-04,4.0,337
2174426,440594,45443053,1660612,2019-12-06,2.0,339
2174427,349309,45443053,1660612,2019-12-07,1.0,340
2174428,637531,45443053,1660612,2019-12-24,17.0,357
2174429,919997,45443053,1660612,2019-12-24,0.0,357
2174430,1214103,45443053,1660612,2019-12-24,0.0,357
2174432,580497,45471482,1945563,2019-02-17,NaN,47
2174433,483565,45471482,1945563,2019-02-27,10.0,57


In [3]:
target.to_csv('targetFor_trainProductUserdf.csv',index=False)

> trainData is seperated last datatime of purchased as a target but ALLdata has all datetimes.

### ---------------------------------------------------------------------------------------

#### Grouping and add some features.

In [4]:
def grouping_addTarget(dataname,data):
    groupeddf = data.groupby(['user_id','product_id'],as_index=False).agg(allDiffDay =('diffDay',lambda x: x.tolist()),
                                                                     LenOfList =('diffDay','size'),
                                                                    allGoneDay=('GoneDays',lambda x: x.tolist()),
                                                                      LastDate = ('created_at','max'))
    
    
    
    groupeddf['ProductProbPerUser'] = groupeddf.groupby('user_id',as_index=False)['LenOfList'].transform(lambda x:x/np.nansum(x))
    
    productProb = pd.DataFrame(data['product_id'].value_counts()).reset_index()
    productProb.columns=['product_id','ProductProb']
    productProb['ProductProb'] = (productProb['ProductProb']/len(productProb['ProductProb']) ).round(4)
    print('productProb: ',productProb.shape)
    display(HTML(productProb.tail(6).to_html()))

    groupeddf=pd.merge(left=groupeddf,right=productProb,how='left',on='product_id')
    
    print('grouped '+dataname+' shape: ',groupeddf.shape)
    with pd.option_context('display.max_rows', 100, 'display.max_columns', 10,'max_colwidth', None):
        display(HTML(groupeddf.tail(6).to_html()))
    
    print('NA in the grouped dataFrame?')
    display(groupeddf.isna().sum())

    return groupeddf

groupedTrainDataset = grouping_addTarget('trainData',trainData)
groupedALLdata = grouping_addTarget('ALLdata',ALLdata)

productProb:  (41127, 2)


,product_id,ProductProb
41121,3639152,0.0001
41122,1867416,0.0001
41123,3457051,0.0001
41124,4256657,0.0001
41125,4623413,0.0001
41126,3760894,0.0001


grouped trainData shape:  (318050, 8)


,user_id,product_id,allDiffDay,LenOfList,allGoneDay,LastDate,ProductProbPerUser,ProductProb
318044,45339741,975982,"[nan, 12.0, 19.0, 19.0, 8.0, 44.0, 36.0, 18.0, 27.0]",9,"[58, 70, 89, 108, 116, 160, 196, 214, 241]",2019-08-30,0.692308,0.0275
318045,45431507,4375282,"[nan, 73.0, 21.0]",3,"[97, 170, 191]",2019-07-11,1.000000,0.1068
318046,45443053,1660612,"[nan, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 7.0, 0.0, 0.0, 0.0, 1.0, 12.0, 6.0, 11.0, 1.0, 1.0, 0.0, 13.0, 0.0, 3.0, 0.0, 2.0, 0.0, 1.0, 3.0, 0.0, 3.0, 4.0, 2.0, 1.0, 1.0, 0.0, 0.0, 1.0, 2.0, 4.0, 2.0, 1.0, 17.0, 0.0, 0.0]",44,"[253, 253, 254, 255, 256, 256, 257, 258, 258, 265, 265, 265, 265, 266, 278, 284, 295, 296, 297, 297, 310, 310, 313, 313, 315, 315, 316, 319, 319, 322, 326, 328, 329, 330, 330, 330, 331, 333, 337, 339, 340, 357, 357, 357]",2019-12-24,1.000000,0.4454
318047,45471482,1945563,"[nan, 10.0, 18.0]",3,"[47, 57, 75]",2019-03-17,0.250000,0.0003
318048,45471482,3309954,"[nan, 8.0, 3.0]",3,"[303, 311, 314]",2019-11-11,0.250000,0.0001
318049,45471482,4563945,"[nan, 3.0, 13.0, 12.0, 12.0, 1.0]",6,"[273, 276, 289, 301, 313, 314]",2019-11-11,0.500000,0.0001


NA in the grouped dataFrame?


...

,product_id,ProductProb
41121,920116,0.0001
41122,1617100,0.0001
41123,2174092,0.0001
41124,932219,0.0001
41125,4470836,0.0001
41126,5417760,0.0001


grouped ALLdata shape:  (318050, 8)


,user_id,product_id,allDiffDay,LenOfList,allGoneDay,LastDate,ProductProbPerUser,ProductProb
318044,45339741,975982,"[nan, 12.0, 19.0, 19.0, 8.0, 44.0, 36.0, 18.0, 27.0, 70.0]",10,"[58, 70, 89, 108, 116, 160, 196, 214, 241, 311]",2019-11-08,0.666667,0.0319
318045,45431507,4375282,"[nan, 73.0, 21.0, 47.0]",4,"[97, 170, 191, 238]",2019-08-27,1.000000,0.1278
318046,45443053,1660612,"[nan, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 7.0, 0.0, 0.0, 0.0, 1.0, 12.0, 6.0, 11.0, 1.0, 1.0, 0.0, 13.0, 0.0, 3.0, 0.0, 2.0, 0.0, 1.0, 3.0, 0.0, 3.0, 4.0, 2.0, 1.0, 1.0, 0.0, 0.0, 1.0, 2.0, 4.0, 2.0, 1.0, 17.0, 0.0, 0.0, 0.0]",45,"[253, 253, 254, 255, 256, 256, 257, 258, 258, 265, 265, 265, 265, 266, 278, 284, 295, 296, 297, 297, 310, 310, 313, 313, 315, 315, 316, 319, 319, 322, 326, 328, 329, 330, 330, 330, 331, 333, 337, 339, 340, 357, 357, 357, 357]",2019-12-24,1.000000,0.5007
318047,45471482,1945563,"[nan, 10.0, 18.0, 146.0]",4,"[47, 57, 75, 221]",2019-08-10,0.266667,0.0003
318048,45471482,3309954,"[nan, 8.0, 3.0, 4.0]",4,"[303, 311, 314, 318]",2019-11-15,0.266667,0.0001
318049,45471482,4563945,"[nan, 3.0, 13.0, 12.0, 12.0, 1.0, 4.0]",7,"[273, 276, 289, 301, 313, 314, 318]",2019-11-15,0.466667,0.0002


NA in the grouped dataFrame?


user_id               0
product_id            0
allDiffDay            0
LenOfList             0
allGoneDay            0
LastDate              0
ProductProbPerUser    0
ProductProb           0
dtype: int64

### ---------------------------------------------------------------------

### Extract some features for each product and for each user. NoWindow aproach.

In [5]:
def extractToApply(x):
    x['avgDiffDay'] = np.nanmean(x['allDiffDay'])
    x['stdDiffDay'] = np.nanstd(x['allDiffDay'])
    x['maxDiffDay'] = np.nanmax(x['allDiffDay'])
    x['minDiffDay'] = np.nanmin(x['allDiffDay'])
    
    x['maxGoneDay'] = np.nanmax(x['allGoneDay'])
    x['minGoneDay'] = np.nanmin(x['allGoneDay'])
    x['medianGoneDay'] = np.nanmedian(x['allGoneDay'])
    x['stdGoneDay'] = np.nanstd(x['allGoneDay'])
    
    return x

#### For "groupedTrainDataset"

In [6]:
import os
workingDirfiles=[]
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        workingDirfiles.append(filename)
workingDirfiles

['targetFor_trainProductUserdf.csv', '__notebook__.ipynb']

In [7]:
fileName = 'trainProductUserdf_addedSomeFeatures.csv'
if fileName in workingDirfiles:
    trainProductUserdf = pd.read_csv('/kaggle/working/'+fileName)
else:
    trainProductUserdf = groupedTrainDataset.apply(extractToApply,axis=1)
print(trainProductUserdf.shape)
with pd.option_context('display.max_rows', 100, 'display.max_columns', 10,'max_colwidth', None):
    display(HTML(trainProductUserdf.tail(5).to_html()))

(318050, 16)


,user_id,product_id,allDiffDay,LenOfList,allGoneDay,LastDate,ProductProbPerUser,ProductProb,avgDiffDay,stdDiffDay,maxDiffDay,minDiffDay,maxGoneDay,minGoneDay,medianGoneDay,stdGoneDay
318045,45431507,4375282,"[nan, 73.0, 21.0]",3,"[97, 170, 191]",2019-07-11,1.00,0.1068,47.000000,26.000000,73.0,21.0,191,97,170.0,40.285095
318046,45443053,1660612,"[nan, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 7.0, 0.0, 0.0, 0.0, 1.0, 12.0, 6.0, 11.0, 1.0, 1.0, 0.0, 13.0, 0.0, 3.0, 0.0, 2.0, 0.0, 1.0, 3.0, 0.0, 3.0, 4.0, 2.0, 1.0, 1.0, 0.0, 0.0, 1.0, 2.0, 4.0, 2.0, 1.0, 17.0, 0.0, 0.0]",44,"[253, 253, 254, 255, 256, 256, 257, 258, 258, 265, 265, 265, 265, 266, 278, 284, 295, 296, 297, 297, 310, 310, 313, 313, 315, 315, 316, 319, 319, 322, 326, 328, 329, 330, 330, 330, 331, 333, 337, 339, 340, 357, 357, 357]",2019-12-24,1.00,0.4454,2.418605,3.871377,17.0,0.0,357,253,311.5,33.147889
318047,45471482,1945563,"[nan, 10.0, 18.0]",3,"[47, 57, 75]",2019-03-17,0.25,0.0003,14.000000,4.000000,18.0,10.0,75,47,57.0,11.585431
318048,45471482,3309954,"[nan, 8.0, 3.0]",3,"[303, 311, 314]",2019-11-11,0.25,0.0001,5.500000,2.500000,8.0,3.0,314,303,311.0,4.642796
318049,45471482,4563945,"[nan, 3.0, 13.0, 12.0, 12.0, 1.0]",6,"[273, 276, 289, 301, 313, 314]",2019-11-11,0.50,0.0001,8.200000,5.114685,13.0,1.0,314,273,295.0,16.326529


In [8]:
#trainProductUserdf.to_csv('trainProductUserdf_addedSomeFeatures.csv',index=False)

#### For "groupedALLdata"

In [9]:
fileName = 'ALLdataProductUserdf_addedSomeFeatures.csv'
if fileName in workingDirfiles:
    ALLdataProductUserdf = pd.read_csv('/kaggle/working/'+fileName)
else:
    ALLdataProductUserdf = groupedALLdata.apply(extractToApply,axis=1)
print(ALLdataProductUserdf.shape)
with pd.option_context('display.max_rows', 100, 'display.max_columns', 10,'max_colwidth', None):
    display(HTML(ALLdataProductUserdf.tail(5).to_html()))

(318050, 16)


,user_id,product_id,allDiffDay,LenOfList,allGoneDay,LastDate,ProductProbPerUser,ProductProb,avgDiffDay,stdDiffDay,maxDiffDay,minDiffDay,maxGoneDay,minGoneDay,medianGoneDay,stdGoneDay
318045,45431507,4375282,"[nan, 73.0, 21.0, 47.0]",4,"[97, 170, 191, 238]",2019-08-27,1.000000,0.1278,47.000000,21.228911,73.0,21.0,238,97,180.5,50.818304
318046,45443053,1660612,"[nan, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 7.0, 0.0, 0.0, 0.0, 1.0, 12.0, 6.0, 11.0, 1.0, 1.0, 0.0, 13.0, 0.0, 3.0, 0.0, 2.0, 0.0, 1.0, 3.0, 0.0, 3.0, 4.0, 2.0, 1.0, 1.0, 0.0, 0.0, 1.0, 2.0, 4.0, 2.0, 1.0, 17.0, 0.0, 0.0, 0.0]",45,"[253, 253, 254, 255, 256, 256, 257, 258, 258, 265, 265, 265, 265, 266, 278, 284, 295, 296, 297, 297, 310, 310, 313, 313, 315, 315, 316, 319, 319, 322, 326, 328, 329, 330, 330, 330, 331, 333, 337, 339, 340, 357, 357, 357, 357]",2019-12-24,1.000000,0.5007,2.363636,3.844068,17.0,0.0,357,253,313.0,33.796881
318047,45471482,1945563,"[nan, 10.0, 18.0, 146.0]",4,"[47, 57, 75, 221]",2019-08-10,0.266667,0.0003,58.000000,62.311048,146.0,10.0,221,47,66.0,70.576200
318048,45471482,3309954,"[nan, 8.0, 3.0, 4.0]",4,"[303, 311, 314, 318]",2019-11-15,0.266667,0.0001,5.000000,2.160247,8.0,3.0,318,303,312.5,5.500000
318049,45471482,4563945,"[nan, 3.0, 13.0, 12.0, 12.0, 1.0, 4.0]",7,"[273, 276, 289, 301, 313, 314, 318]",2019-11-15,0.466667,0.0002,7.500000,4.924429,13.0,1.0,318,273,301.0,17.235464


In [10]:
#ALLdataProductUserdf.to_csv('ALLdataProductUserdf_addedSomeFeatures.csv',index=False)

### ----------------------------------------------------------------------------------

#### convert Gone day of a year to datetime format. start from 2019-01-01

In [11]:
def convert_goneDayTo_datetime(data,GoneDayColumn,startFrom='2019-01-01'):
    return pd.to_datetime(data[GoneDayColumn],unit='D',origin=startFrom).dt.strftime('%Y-%m-%d')

#### Add Dummy prediction by (maxGoneDay+avgDiffDay)

### trainProductUserdf

In [12]:
trainProductUserdf['dummyPrediction_goneDay'] = trainProductUserdf['maxGoneDay']+trainProductUserdf['avgDiffDay']
trainProductUserdf['dummyPrediction_datetime']= convert_goneDayTo_datetime(trainProductUserdf,'dummyPrediction_goneDay')
with pd.option_context('display.max_rows', 100, 'display.max_columns', 10,'max_colwidth', None):
    display(HTML(trainProductUserdf.tail(2).to_html()))

,user_id,product_id,allDiffDay,LenOfList,allGoneDay,LastDate,ProductProbPerUser,ProductProb,avgDiffDay,stdDiffDay,maxDiffDay,minDiffDay,maxGoneDay,minGoneDay,medianGoneDay,stdGoneDay,dummyPrediction_goneDay,dummyPrediction_datetime
318048,45471482,3309954,"[nan, 8.0, 3.0]",3,"[303, 311, 314]",2019-11-11,0.25,0.0001,5.5,2.500000,8.0,3.0,314,303,311.0,4.642796,319.5,2019-11-16
318049,45471482,4563945,"[nan, 3.0, 13.0, 12.0, 12.0, 1.0]",6,"[273, 276, 289, 301, 313, 314]",2019-11-11,0.50,0.0001,8.2,5.114685,13.0,1.0,314,273,295.0,16.326529,322.2,2019-11-19


In [13]:
trainProductUserdf.to_csv('trainProductUserdf_addedSomeFeatures_addedDummyPrediction.csv',index=False)

### ALLdataProductUserdf

In [14]:
ALLdataProductUserdf['dummyPrediction_goneDay'] = ALLdataProductUserdf['maxGoneDay']+ALLdataProductUserdf['avgDiffDay']
ALLdataProductUserdf['dummyPrediction_datetime']= convert_goneDayTo_datetime(ALLdataProductUserdf,'dummyPrediction_goneDay')
with pd.option_context('display.max_rows', 100, 'display.max_columns', 10,'max_colwidth', None):
    display(HTML(ALLdataProductUserdf.tail(2).to_html()))

,user_id,product_id,allDiffDay,LenOfList,allGoneDay,LastDate,ProductProbPerUser,ProductProb,avgDiffDay,stdDiffDay,maxDiffDay,minDiffDay,maxGoneDay,minGoneDay,medianGoneDay,stdGoneDay,dummyPrediction_goneDay,dummyPrediction_datetime
318048,45471482,3309954,"[nan, 8.0, 3.0, 4.0]",4,"[303, 311, 314, 318]",2019-11-15,0.266667,0.0001,5.0,2.160247,8.0,3.0,318,303,312.5,5.500000,323.0,2019-11-20
318049,45471482,4563945,"[nan, 3.0, 13.0, 12.0, 12.0, 1.0, 4.0]",7,"[273, 276, 289, 301, 313, 314, 318]",2019-11-15,0.466667,0.0002,7.5,4.924429,13.0,1.0,318,273,301.0,17.235464,325.5,2019-11-22


In [15]:
ALLdataProductUserdf.to_csv('ALLdataProductUserdf_addedSomeFeatures_addedDummyPrediction.csv',index=False)

### --------------------------------------------------------------------

Now our task is make a model to predict next datetime that a customer will buy a specific product.\
We have two aproach to feature ectraction -> noWindow and window appraoches.\
After feature extraction -> make a prediction model to predict goneDay from base datetime(2019-01-01).

NoWindow feature extraction is done in this notebook.\
Create window approach for additional features see this paper([link](https://www.researchgate.net/publication/346846529_Using_machine_learning_to_predict_the_next_purchase_date_for_an_individual_retail_customer)),these features will use on window version datetime presiction.

Continue the codes in the next notebooks.